<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>

# 0.0 Automatically Generate Synthetic Dataset

In this notebook, you'll explore the [Omniverse Replicator](https://docs.omniverse.nvidia.com/prod_extensions/prod_extensions/ext_replicator.html) API and create your own synthetic dataset with a script in "headless" mode.

**[0.1 Learning Objectives](#0.1-Learning-Objectives)<br>**
**[0.2 Omniverse Code GUI Demo](#0.-Omniverse-Code-GUI-Demo)<br>**
**[0.3 Walk Through Our Replicator Script](#0.3-Walk-Through-Our-Replicator-Script)<br>**
**[0.4 Running the Replicator Script](#0.4-Running-the-Replicator-Script)<br>**
&nbsp;&nbsp;&nbsp;&nbsp;[0.4.1 Bonus Replicator API Work](#0.4.1-Bonus-Replicator-API-Work)<br>

---
## 0.1 Learning Objectives

In this first notebook we will step through each piece of a Replicator script to create synthetic data. At the end of the notebook, you'll run the data generation script.  

This notebook introduces:
- **Omniverse Replicator API:** - Python API for generating physically accurate synthetic data. ([Replicator Core API documentation](https://docs.omniverse.nvidia.com/py/replicator/1.6.4/source/extensions/omni.replicator.core/docs/index.html))
- **Omniverse Replicator Headless:** - Running Replicator scripts without using the Omniverse GUI. ([Running Replicator Headlessly documentation](https://docs.omniverse.nvidia.com/app_code/prod_extensions/ext_replicator/headless_example.html))

---
## 0.2 Omniverse Code GUI Demo

Before walking through the data generation script, watch this video to learn how this could be done in the Omniverse Code GUI. Omniverse is installed on the VM Desktop if you would like to experiment for yourself!

<center><video controls src="https://dli-lms.s3.amazonaws.com/assets/s-ov-10-v1/DLI_part_2.mp4" width=800 ></center>

---
## 0.3 Walk Through Our Replicator Script

Now that you've seen how to run the script from the GUI, watch this quick intro video on running the script headlessly. 

Remember, you will only be running the script at the end of the notebook; the rest is a guided walkthrough of the Replicator API script we are using.

<center><video controls src="https://dli-lms.s3.amazonaws.com/assets/s-ov-10-v1/DLI_part_3.mp4" width=800 ></center>

Using the Python Replicator API, we will generate the RGB images and labeled bounding box data that we need to fine-tune a pretrained model later.

We utilize the current `datetime` at the end of this Python script to add the current date to our output data folder. That way, if we end up generating multiple datasets we can easily tell them apart.

The `omni.replicator.core` import is the Replicator API functionality that we are pulling from Omniverse. We will see lots of Replicator-specific syntax throughout this script. 

```python
import datetime
now = datetime.datetime.now()
import omni.replicator.core as rep
```

The first thing we do is create a new [universal scene description (USD) layer](https://docs.omniverse.nvidia.com/extensions/latest/ext_layers.html). A layer in this context is used to organize our scene. We are working with a single layer for this example.

Next, we define the 3D objects that will make up our scene. The paths here correspond to the location of the USD in the shared Omniverse drive on the Code GUI as we saw in the original GUI demo.

The `FRUIT_PROPS` outlines all the fruits we will be filling our crate with, you will see as we continue how we randomize the fruits to make each frame we generate unique. 

```python
with rep.new_layer():
    CRATE = 'omniverse://ove-nucleus.courses.nvidia.com/NVIDIA/Samples/Marbles/assets/standalone/SM_room_crate_3/SM_room_crate_3.usd'
    SURFACE = 'omniverse://ove-nucleus.courses.nvidia.com/NVIDIA/Assets/Scenes/Templates/Basic/display_riser.usd'
    ENVS = 'omniverse://ove-nucleus.courses.nvidia.com/NVIDIA/Assets/Scenes/Templates/Interior/ZetCG_ExhibitionHall.usd'
    FRUIT_PROPS = {
        'apple': 'omniverse://ove-nucleus.courses.nvidia.com/NVIDIA/Assets/ArchVis/Residential/Food/Fruit/Apple.usd',
        'avocado': 'omniverse://ove-nucleus.courses.nvidia.com/NVIDIA/Assets/ArchVis/Residential/Food/Fruit/Avocado01.usd',
        'kiwi': 'omniverse://ove-nucleus.courses.nvidia.com/NVIDIA/Assets/ArchVis/Residential/Food/Fruit/Kiwi01.usd',
        'lime': 'omniverse://ove-nucleus.courses.nvidia.com/NVIDIA/Assets/ArchVis/Residential/Food/Fruit/Lime01.usd',
        'lychee': 'omniverse://ove-nucleus.courses.nvidia.com/NVIDIA/Assets/ArchVis/Residential/Food/Fruit/Lychee01.usd',
        'pomegranate': 'omniverse://ove-nucleus.courses.nvidia.com/NVIDIA/Assets/ArchVis/Residential/Food/Fruit/Pomegranate01.usd',
        'onion': 'omniverse://ove-nucleus.courses.nvidia.com/NVIDIA/Assets/ArchVis/Residential/Food/Vegetables/RedOnion.usd',
        'strawberry': 'omniverse://ove-nucleus.courses.nvidia.com/NVIDIA/Assets/ArchVis/Residential/Food/Berries/strawberry.usd',
        'lemon': 'omniverse://ove-nucleus.courses.nvidia.com/NVIDIA/Assets/ArchVis/Residential/Decor/Tchotchkes/Lemon_01.usd',
        'orange': 'omniverse://ove-nucleus.courses.nvidia.com/NVIDIA/Assets/ArchVis/Residential/Decor/Tchotchkes/Orange_01.usd'    }
```

The next portion of this script will define our randomization functions. The first of which randomizes the fruits in the crate. Below we will define what key lines are doing:

 - ```instances = rep.randomizer.instantiate(file_name, size=max_number, mode='scene_instance')```
    - Create `scene_instances` from fruits list so we can use the [physics properties](https://docs.omniverse.nvidia.com/prod_extensions/prod_extensions/ext_replicator/physics_example.html#using-the-replicator-apis) supplied by Replicator.
 - ```rep.modify.pose( ... )```
    - Here we define the distribution of locations for the fruits to be placed, all fall within the range of the crate. We add more variation by allowing the scale of the fruit to change. 
 - ```rep.modify.visibility( ... )```
    - Finally we are varying the visibility of the fruit so that each image doesn't have all the fruits from the list.

```python
 def random_props(file_name, class_name, max_number=1, one_in_n_chance=3):
        instances = rep.randomizer.instantiate(file_name, size=max_number, mode='scene_instance')
        with instances:
            # First implementation of collisions for Replicator, further work coming to keep objects from overlapping
            rep.physics.collider()
            rep.modify.semantics([('class', class_name)])
            rep.modify.pose(
                position=rep.distribution.uniform((-4, 4, -25), (4, 15, 25)),
                rotation=rep.distribution.uniform((-180,-180, -180), (180, 180, 180)),
                scale = rep.distribution.uniform((0.8), (1.2)),
            )
            rep.modify.visibility(rep.distribution.choice([True],[False]*(one_in_n_chance)))
        return instances.node
```

In a similar fashion we will randomize the the lighting in each frame. This will add more variation to our dataset and will more closely reflect the different conditions we might see in the real world.

We create sphere lights where each available property is altered. The positions, scale, and look of the light is randomized for each image. 

```python
def sphere_lights(num):
    lights = rep.create.light(
        light_type="Sphere",
        temperature=rep.distribution.normal(3000, 500),
        intensity=rep.distribution.normal(30000, 1000),
        position=rep.distribution.uniform((-300, -300, -300), (300, 300, 300)),
        scale=rep.distribution.uniform(50, 100),
        count=num        )
    return lights.node
```

Now we register these randomization functions for our Replicator script.

```python
rep.randomizer.register(random_props)
rep.randomizer.register(sphere_lights)
```

To set up our static elements, we use `rep.create.from_usd` for the environment, crate, and surface. These will remain static for every frame. For the surface we are enabling the `physics.collider` so that our crate will fall onto the surface.

For the crate we add some additional information. We again add the `physics.collider` and also give the crate a weight. The position of the crate is also statically defined to the center of our surface.

At this phase all of our static elements are set in the scene. 



```python
env = rep.create.from_usd(ENVS)
surface = rep.create.from_usd(SURFACE)
with surface:
    rep.physics.collider()
crate = rep.create.from_usd(CRATE)
with crate:
    rep.physics.collider()
    rep.physics.mass(mass=100)
    rep.modify.pose(
            position=(0,20,0),
            rotation=(0, 0, 90)
        )
```

For every Replicator script we write we will want to create a camera that looks at our scene. We then attach the camera to a render product. This combination allows for the viewport by which each frame will be generated. 

```python
camera =  rep.create.camera()
render_product = rep.create.render_product(camera, resolution=(1024, 1024))
```

At this point we are looking to start the randomization for each frame. For this use case we are generating 100 data frames. First we loop through the fruit we defined and utilize the randomize function.  We also run the light randomization function. 

The final step here is to modify the position of our camera. Instead of showing the crate from the same angle for every frame we use the camera position to simulate crate movement. The key here is `look_at` which ensures that even as we move the camera we continue to look at the crate position.

```python
with rep.trigger.on_frame(num_frames=100):
    rep.modify.timeline(5, "frame")
    for n, f in FRUIT_PROPS.items():
        random_props(f, n)
    rep.randomizer.sphere_lights(5)
    with camera:
        rep.modify.pose(position=rep.distribution.uniform((-20, 90, -17), (10, 140, -15)), look_at=(0,20,0))
```

Finally, for this sample we utilize the built in [basic writer](https://docs.omniverse.nvidia.com/extensions/latest/ext_replicator/writer_examples.html#basic-writer). We attach this writer to the render product to produce the dataset output. For our training we want to have the original images as well as the bounding box coordinates and label annotations. 

We utilize the date information we imported at the top of the script to save our data to a specific location.

```python
writer = rep.WriterRegistry.get("BasicWriter")
now=now.strftime("%Y-%m-%d_%H:%M:%S")
output_dir = "/dli/task/data/fruit_data_"+now
writer.initialize( output_dir=output_dir, rgb=True, bounding_box_2d_tight=True)
writer.attach([render_product])
```

Behind the scenes, Replicator is creating an [OmniGraph](https://docs.omniverse.nvidia.com/prod_extensions/prod_extensions/ext_omnigraph.html#omnigraph-short) which Omniverse will execute. This line does the execution and without it we would not get any ouput.

```python
rep.orchestrator.run()
```

---
## 0.4 Running the Replicator Script

Now let's run our data generation script that we have just stepped through! Copy the command from below (starting at `cd` and ending at `.py`) and paste it into a terminal. You can get a new terminal by clicking the blue `+` and then `Terminal`. Note: your first time running will take Replicator some time to run.  You may see a number of warnings and even errors in the output of terminal window.  When the replicator script is complete, you'll receive

Note: your first time running will take Replicator several minutes to complete (estimate 10 minutes).  You may see a number of warnings and even errors in the output of terminal window during the script execution.  When the replicator script is complete, you'll receive a notice that the application is stopping and the terminal prompt will appear:

```yaml
[C 2023-04-19 20:28:29.590 ServerApp] received signal 15, stopping
[I 2023-04-19 20:28:29.591 ServerApp] Shutting down 3 extensions
[I 2023-04-19 20:28:29.591 ServerApp] Shutting down 0 terminals
user@ip:~/.local/share/ov/pkg/code-2022.3.1$ 
```

Your new output is in the `data` directory.  There should be a new folder, `fruit_data_$DATE` containing the generated data, including some `.png` files you can open and view with a simple double-click.

### 0.4.1 Bonus Replicator API Work

If you are looking for some bonus work, navigate to the Replicator script you ran above `/opt/project/generate_data_headless.py` and open in Visual Studio Code. From there you can make your own edits. Some suggestions that won't effect the training are, changing your scene, changing the environment, changing the lighting randomization. Though these changes can be made in the python script, you may find working with the script in the Omniverse Code GUI to be useful. When you are ready to test your changes either copy to the `Script Editor` in Omniverse Code or rerun the command above.

---
<h2 style="color:green;">Congratulations!</h2>

In this notebook, you have:
- Learned how the Omniverse Code GUI can be used to start generating synthetic data
- Explored and run a basic Replicator Python script
- Generated a synthetic dataset headlessly through the terminal

When you are ready to move on to using your dataset for training, close this window and click the "Training Lab" link on the remote desktop.

<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>